# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Oct  7 2022  9:58AM  248148        15  ALIL355676-01   
1  Oct  7 2022  9:58AM  248148        15  ALI4578462234   
2  Oct  7 2022  9:58AM  248148        15       30011277   
3  Oct  7 2022  9:58AM  248148        15       30011278   
4  Oct  7 2022  9:58AM  248148        15       30011281   
5  Oct  7 2022  9:58AM  248148        15       30011280   
6  Oct  7 2022  9:58AM  248148        15       30011279   
7  Oct  7 2022  9:58AM  248148        15       30011282   
8  Oct  7 2022  9:58AM  248148        15       30011284   
9  Oct  7 2022  9:58AM  248148        15       30011283   

                Customer ShipmentStatus  
0  Alliance Pharma, Inc.       Released  
1  Alliance Pharma, Inc.       Released  
2  Alliance Pharma, Inc.       Released  
3  Alliance Pharma, Inc.       Released  
4  Alliance Pharma, Inc.       Released  
5  Alliance Pharma, Inc.       Released  
6  Alliance Pharma, Inc.       Released  
7  Alliance Pharma, Inc.       Released  
8  Alliance Pharma, Inc.       Released  
9  Alliance Pharma, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
20  248139       Released          2
21  248140       Released         59
22  248141       Released          8
23  248145       Released          1
24  248148       Released         11

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
248139                NaN        NaN       2.0
248140                NaN        NaN      59.0
248141                NaN        NaN       8.0
248145                NaN        NaN       1.0
248148                NaN        NaN      11.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248037                7.0        0.0       1.0
248074                0.0        0.0      42.0
248085                0.0        0.0       1.0
248097                0.0        1.0       0.0
248115                0.0        0.0       9.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248037                  7          0         1
248074                  0          0        42
248085                  0          0         1
248097                  0          1         0
248115                  0          0         9

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               248037          7          0         1
1               248074          0          0        42
2               248085          0          0         1
3               248097          0          1         0
4               248115          0          0         9

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               248037         7                  1
1               248074                           42
2               248085                            1
3               248097                   1         
4               248115                            9

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                            Customer
0    Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.
11   Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.
12   Oct  7 2022  9:25AM  248141        10                             Bio-PRF
20   Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation
79   Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service
81   Oct  7 2022  8:50AM  248135        10                   ISDIN Corporation
102  Oct  7 2022  8:30AM  248118        10                      Citieffe, Inc.
103  Oct  6 2022  6:02PM  248074        15               Alliance Pharma, Inc.
145  Oct  6 2022  5:44PM  248127        21                   NBTY Global, Inc.
146  Oct  6 2022  5:42PM  248126        21                   NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                            Customer  \
0  Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.   
1  Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.   
2  Oct  7 2022  9:25AM  248141        10                             Bio-PRF   
3  Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation   
4  Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service   
5  Oct  7 2022  8:50AM  248135        10                   ISDIN Corporation   
6  Oct  7 2022  8:30AM  248118        10                      Citieffe, Inc.   
7  Oct  6 2022  6:02PM  248074        15               Alliance Pharma, Inc.   
8  Oct  6 2022  5:44PM  248127        21                   NBTY Global, Inc.   
9  Oct  6 2022  5:42PM  248126        21                   NBTY Global, Inc.   

  Completed Executing Released  
0                           11  
1                            1  
2                            8  
3                           59  
4                            2  
5                   6       15  
6                            1  
7                           42  
8                            1  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                            Customer  \
0  Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.   
1  Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.   
2  Oct  7 2022  9:25AM  248141        10                             Bio-PRF   
3  Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation   
4  Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service   
5  Oct  7 2022  8:50AM  248135        10                   ISDIN Corporation   
6  Oct  7 2022  8:30AM  248118        10                      Citieffe, Inc.   
7  Oct  6 2022  6:02PM  248074        15               Alliance Pharma, Inc.   
8  Oct  6 2022  5:44PM  248127        21                   NBTY Global, Inc.   
9  Oct  6 2022  5:42PM  248126        21                   NBTY Global, Inc.   

  Released Executing Completed  
0       11                      
1        1                      
2        8                      
3       59                      
4        2                      
5       15         6            
6        1                      
7       42                      
8        1                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.   
1  Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.   
2  Oct  7 2022  9:25AM  248141        10                             Bio-PRF   
3  Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation   
4  Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service   

  Released Executing Completed  
0       11                      
1        1                      
2        8                      
3       59                      
4        2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.   
1  Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.   
2  Oct  7 2022  9:25AM  248141        10                             Bio-PRF   
3  Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation   
4  Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service   

   Released  Executing  Completed  
0      11.0        NaN        NaN  
1       1.0        NaN        NaN  
2       8.0        NaN        NaN  
3      59.0        NaN        NaN  
4       2.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                            Customer  \
0  Oct  7 2022  9:58AM  248148        15               Alliance Pharma, Inc.   
1  Oct  7 2022  9:30AM  248145        10                      Citieffe, Inc.   
2  Oct  7 2022  9:25AM  248141        10                             Bio-PRF   
3  Oct  7 2022  9:13AM  248140        10                   ISDIN Corporation   
4  Oct  7 2022  9:01AM  248139        16  Sartorius Lab Products and Service   

   Released  Executing  Completed  
0      11.0        0.0        0.0  
1       1.0        0.0        0.0  
2       8.0        0.0        0.0  
3      59.0        0.0        0.0  
4       2.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1240679      84.0        6.0        0.0
15          744259      54.0        0.0        7.0
16         2977444      46.0        2.0        0.0
20          248085       1.0        0.0        0.0
21          496253       1.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1240679      84.0        6.0        0.0
1        15   744259      54.0        0.0        7.0
2        16  2977444      46.0        2.0        0.0
3        20   248085       1.0        0.0        0.0
4        21   496253       1.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      84.0        6.0        0.0
1        15      54.0        0.0        7.0
2        16      46.0        2.0        0.0
3        20       1.0        0.0        0.0
4        21       1.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   84.0
1        15  Released   54.0
2        16  Released   46.0
3        20  Released    1.0
4        21  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    16   20   21
Status                               
Completed   0.0   7.0   0.0  0.0  0.0
Executing   6.0   0.0   2.0  0.0  1.0
Released   84.0  54.0  46.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    16   20   21
0          Completed   0.0   7.0   0.0  0.0  0.0
1          Executing   6.0   0.0   2.0  0.0  1.0
2           Released  84.0  54.0  46.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    16   20   21
0  Completed   0.0   7.0   0.0  0.0  0.0
1  Executing   6.0   0.0   2.0  0.0  1.0
2   Released  84.0  54.0  46.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()